# Example Usage

This is a basic example using the torchvision COCO dataset from coco.py, it assumes that you've already downloaded the COCO images and annotations JSON.  You'll notice that the scale augmentations are quite extreme.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import cv2
import numpy as np
from copy_paste import CopyPaste
from coco import CocoDetectionCP
from visualize import display_instances
import albumentations as A
import random
from matplotlib import pyplot as plt

In [5]:
transform = A.Compose([
        A.RandomScale(scale_limit=(-0.9, 1), p=1), #LargeScaleJitter from scale of 0.1 to 2
        A.PadIfNeeded(256, 256, border_mode=0), #pads with image in the center, not the top left like the paper
        A.RandomCrop(256, 256),
        CopyPaste(blend=True, sigma=1, pct_objects_paste=0.8, p=1.) #pct_objects_paste is a guess
    ], bbox_params=A.BboxParams(format="coco", min_visibility=0.05)
)

data = CocoDetectionCP(
    '../../datasets/coco/train2014/', 
    '../../datasets/coco/annotations/instances_train2014.json', 
    transform
)

ModuleNotFoundError: No module named 'pycocotools'

In [13]:
import torch
import cv2
import numpy as np
import albumentations as A
from matplotlib import pyplot as plt

# Load your data
data = torch.load("ganglia_test/has_269_S16-1197 B3.pt")

# Extract images and maps, and convert them to numpy for visualization and processing
i = data['imgs'].squeeze(dim=0).numpy().transpose(1, 2, 0)  # Convert from torch to numpy and CHW to HWC format
p = data['ganglia_potential_map'].squeeze(dim=0).numpy()
c = data['ganglia_certain_map'].squeeze(dim=0).numpy()


# Define the transformation pipeline (excluding CopyPaste for simplicity)
transform = A.Compose([
    A.RandomScale(scale_limit=(-0.9, 1), p=1),
    A.PadIfNeeded(256, 256, border_mode=0),
    A.RandomCrop(256, 256),
    CopyPaste(blend=True, sigma=1, pct_objects_paste=0.8, p=1.)
    # CopyPaste could be included here if applicable
])

# Apply transformations
transformed = transform(image=i)
transformed_image = transformed["image"]

# Visualization
f, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(i)
ax[0].set_title('Original Image')
ax[1].imshow(transformed_image)
ax[1].set_title('Transformed Image')
plt.show()

TypeError: super(type, obj): obj must be an instance or subtype of type

In [14]:
import cv2
import numpy as np
import albumentations as A
from copy import deepcopy
from skimage.filters import gaussian

def image_copy_paste(img, paste_img, alpha, blend=True, sigma=1):
    if alpha is not None:
        if blend:
            alpha = gaussian(alpha, sigma=sigma, preserve_range=True)

        img_dtype = img.dtype
        alpha = alpha[..., None]  # Ensure alpha channel is correctly shaped
        img = paste_img * alpha + img * (1 - alpha)
        img = img.astype(img_dtype)

    return img

class CopyPaste(A.DualTransform):
    def __init__(
        self,
        blend=True,
        sigma=3,
        p=0.5,
        always_apply=False
    ):
        super(CopyPaste, self).__init__(always_apply, p)
        self.blend = blend
        self.sigma = sigma

    def apply(self, img, paste_img=None, alpha=None, **params):
        """Applies the copy-paste augmentation."""
        if paste_img is None or alpha is None:
            return img  # Return original if no paste_img or alpha provided
        return image_copy_paste(img, paste_img, alpha, blend=self.blend, sigma=self.sigma)

    def get_transform_init_args_names(self):
        return ("blend", "sigma")

    def get_params_dependent_on_targets(self, params):
        # Example: Extracting `paste_img` and `alpha` from params
        paste_img = params.get('paste_img')
        alpha = params.get('alpha')
        return {"paste_img": paste_img, "alpha": alpha}

    def apply_to_mask(self, mask, **params):
        # Modify if you want to apply something specific to masks
        return mask

# Example usage with your data
transform = A.Compose([
    CopyPaste(p=1.0)  # Adjust parameters as needed
], additional_targets={'paste_img': 'image', 'alpha': 'mask'})  # Define additional targets if needed

# Assuming you have 'i' (image), 'p' (potential map), and 'c' (certain map) loaded
# You would need to adjust how you create `paste_img` and `alpha` for your specific use case
# Example: creating an alpha mask based on 'p' or 'c', and defining a paste_img

# Applying transformation
# This is an illustrative example, adjust 'image', 'paste_img', and 'alpha' as per your data
transformed = transform(image=i, paste_img=i, alpha=p)
transformed_image = transformed["image"]

# Visualization (Adjust as per your previous visualization code)


In [17]:
import torch
import cv2
import numpy as np
import albumentations as A
from matplotlib import pyplot as plt

# Load your data
data = torch.load("ganglia_test/has_269_S16-1197 B3.pt")

# Extract images and maps, and convert them to numpy for visualization and processing
i = data['imgs'].squeeze(dim=0).numpy().transpose(1, 2, 0)  # Convert from torch to numpy and CHW to HWC format
p = data['ganglia_potential_map'].squeeze(dim=0).numpy()
c = data['ganglia_certain_map'].squeeze(dim=0).numpy()


# Define the transformation pipeline (excluding CopyPaste for simplicity)
transform = A.Compose([
    A.RandomScale(scale_limit=(-0.9, 1), p=1),
    A.PadIfNeeded(256, 256, border_mode=0),
    A.RandomCrop(256, 256),
    CopyPaste(blend=True, sigma=1, pct_objects_paste=0.8, p=1.)
    # CopyPaste could be included here if applicable
])

# Apply transformations
transformed = transform(image=i)
transformed_image = transformed["image"]

# Visualization
f, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(i)
ax[0].set_title('Original Image')
ax[1].imshow(transformed_image)
ax[1].set_title('Transformed Image')
plt.show()

[autoreload of copy_paste failed: Traceback (most recent call last):
  File "/home/anthony/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/anthony/anaconda3/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/anthony/anaconda3/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/anthony/Desktop/Hirsch_Research/copy_paste.py", line 61, in <module>
    transformed = transform(image=i, paste_img=i, alpha=p)
                                  ^
NameError: name 'i' is not defined
]


TypeError: CopyPaste.__init__() got an unexpected keyword argument 'pct_objects_paste'